In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

################################################################################
##########################SET VARIABLES HERE####################################

YEARS = ['2020-2021', '2021-2022']      ## Options include: '2020-2021' OR '2021-2022'
QUARTERS = ['Fall', 'Winter', 'Spring']                     ## Options include: 'Fall' OR 'Winter' OR 'Spring' OR 'Summer'
EVENTS = ['Workshops']   ## Options include: 'Chem Virtual Study Halls' OR 'ENG Grad Studios' OR 'Math Virtual Study Halls' OR 'Virtual Study Halls' OR 'Workshops' OR 'Grad Studios' OR 'Study Halls' OR 'Workshops' OR 'Chem 31B Study Halls' OR 'Studios' OR 'In-Person Study Halls'
TARGET_METRIC = 'Stanford School'      ## Options include: 'Year at Stanford' OR 'Major/Degree' OR 'Stanford School' OR 'Role at Stanford'
TYPE_OF_DISPLAY = 'All Visits'       ## Options include: 'Unique Visits' OR 'All Visits'
ATTENDEE_TYPE = 'All Students'          ## Options include: 'Graduate Students' OR 'Undergraduate Students' OR 'All Students'

################################################################################


################################################################################
#####MOUNTS GOOGLE DRIVE; UNCOMMENT NEXT TWO LINES AND RUN ONCE PER SESSION#####

from google.colab import drive
drive.mount('/content/drive')
################################################################################

#translate TARGET_METRIC to COLUMN_OF_INTEREST
COLUMN_OF_INTEREST = TARGET_METRIC

if COLUMN_OF_INTEREST == 'Major/Degree':
  if ATTENDEE_TYPE == 'Undergraduate Students':
    COLUMN_OF_INTEREST = 'Undergraduate Major'
  if ATTENDEE_TYPE == 'Graduate Students':
    COLUMN_OF_INTEREST = 'Degree Program:' 

# creates empty data frame used to merge all target data into
merged_target_df = pd.DataFrame()

#loops through specified search parameters
for YEAR in YEARS:
  for QUARTER in QUARTERS: 
    for EVENT in EVENTS:
      #sets one search path of interest
      current_path =  '/content/drive/My Drive/NS CTL Data Visualization/Raw Data/' + YEAR + '/' + QUARTER + '/' + EVENT
      # narrow down files to just excels
      filenames = glob.glob(current_path + '/*.xlsx')
      if filenames == []:
        # produces warning when data of interest is not found within a search parameter
        print(EVENT + " data not found in " + QUARTER + " " +YEAR)
      else: 
        for file in filenames: 
        # iterates through excel files and append relevant info to merged_target_df
          next_all_df = pd.read_excel(file, sheet_name= 0)
          next_all_df['Full Name'] = next_all_df['First Name'] + next_all_df['Last Name']

          if COLUMN_OF_INTEREST == 'Role at Stanford':
            if 'Which degree(s) are you pursuing?' in next_all_df: 
              COLUMN_OF_INTEREST = 'Which degree(s) are you pursuing?'
            elif 'Role at Stanford' in next_all_df: 
              COLUMN_OF_INTEREST = 'Role at Stanford'
            else:
              break


          # handle exception for Major/Degree metric, which analyzes two different columns for Undergrads and Grads, if specified for All Students
          if TARGET_METRIC == 'Major/Degree' and ATTENDEE_TYPE == 'All Students':
            if 'Degree Program:' in next_all_df and 'Undergraduate Major' in next_all_df:
              COLUMN_OF_INTEREST = 'Major/Degree'
              next_all_df['Major/Degree'] = next_all_df['Undergraduate Major'].fillna('').astype(str) + next_all_df['Degree Program:'].fillna('').astype(str)
            elif 'Undergraduate Major' in next_all_df:
              COLUMN_OF_INTEREST = 'Undergraduate Major'
            else:
              COLUMN_OF_INTEREST = 'Degree Program:'
              
          next_target_df = next_all_df[['Full Name', 'Ticket Type', COLUMN_OF_INTEREST]].copy()
          merged_target_df = merged_target_df.append(next_target_df, ignore_index = True)

if merged_target_df.empty: 
  # produces warning when data of interest is not found within ANY search parameter
  print('no data found within search parameters')
else: 
  if TYPE_OF_DISPLAY == 'Unique Visits':
    # filters out duplicate attendee data if specified
    merged_target_df = merged_target_df.drop_duplicates(subset=['Full Name'])

  if ATTENDEE_TYPE != 'All Students':
    # can be used to further filter out data (i.e. by Undergrad vs Grad, etc.), uncomment to use 
    merged_target_df = merged_target_df[merged_target_df["Ticket Type"] == ATTENDEE_TYPE]

  # separate merged column of interest into col_df
  col_df = merged_target_df[COLUMN_OF_INTEREST]

  if TARGET_METRIC == 'Year at Stanford':
    #used when searching Year at Stanford to  strip extra spaces
    col_df = col_df.fillna('No Info').astype(str)
    col_df = col_df.replace(' ', '')
    col_df = col_df.replace('1.0', '1')
    col_df = col_df.replace('2.0', '2')
    col_df = col_df.replace('3.0', '3')
    col_df = col_df.replace('4.0', '4')
    col_df = col_df.replace('5.0', '5+')
  count = col_df.value_counts() ##if count zero
  #turn count Series to df and change column names
  count_df = pd.DataFrame({TARGET_METRIC:count.index, '# of Students':count.values})

  #saves csv to your personal drive in a folder labeled "New Processed Data", all original processed files in All Processed Data, uncomment to use for debugging
  #count_df.to_csv('/content/drive/My Drive/NS CTL Data Visualization/Winter and Spring Quarter 2020-2021 ENG Grad Studios-year.csv', index=False)
  
  if count.empty:
    #produces warning when data of interest is not found with ANY specified parameters
    print('no data found within specified parameters')
  else: 
    count_df.plot(x = TARGET_METRIC, y='# of Students', kind = 'bar')

